# Can you predict the Tide?

## Goal

Participants will have to forecast the sea surges in two western European coastal cities.

We place ourselves in a forecast setup: knowing the surge values and the sea-level pressure field in the last 5 days, we want to predict the surge values in the next five days. It is hence a time series prediction problem. The signals we consider are:

- the surge, which is a function of the time.
- the sea-level pressure, which is a function of the time, the latitude and the longitude.

The score $l(\hat{y},y)$ we use to measure the quality of the prediction $\hat{y}$​ compared to the true values $y$ is a weighted version of the mean square error (MSE). The weights depend linearly on the forecast time, with a bigger weight for the first forecast time and a lower weight for the last forecast time. The prediction for the two cities are computed independently, and the final loss is their sum:

```python
def surge_prediction_metric(y_true, y_pred):
    w = np.linspace(1, 0.1, 10)[np.newaxis]
    surge1_cols = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9' ]
    surge2_cols = [
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_score = (w * (y_true[surge1_cols].values - y_pred[surge1_cols].values)**2).mean()
    surge2_score = (w * (y_true[surge2_cols].values - y_pred[surge2_cols].values)**2).mean()

    return surge1_score + surge2_score
```


Since the surge values are normalized (zero mean and standard deviation 1), $1-l$ can be seen as a percentage of explained variance. With a trivial zero prediction of all values, the score is $l \approx 1$ , meaning that we explain 0 % of the variance. A score bigger than one is hence worse that the zero prediction and can be considered as "bad".

## Data description

The training set contains 5599 entries, and the test set contains 509 entries. Each entry represents approximately 5 days of measurements of the pressure and the tide, and the times at which they were done.

**Times** are given in the GMT convention. In the GMT convention, the time is expressed as the number of seconds elapsed since January 1st, 1970; they can be converted to the usual Gregorian time with `time.gmtime()`. Note that in our dataset, some times are negative: indeed, the first measurements date back to the 1950s. For instance, the very first sea-level pressure field is given at $t=−631108800.0$ which corresponds to 1950, January 1st at approximately 21h.

**Sea-Level** Pressure fields (SLP) are given every three hours, so there are 40 fields for every observation. They cover the Atlantic front of western Europe and Iceland, as shown on the following map:

![SLP map](images/slp.png)

Sea surges are measured for each high tide, i.e. every 12 hours approximately. We measure the sea surge at two different locations, which are two anonymous European costal cities. Consequently, each entry contains `2×102×10` values to predict. There are in total $20×509=10180$ values to predict for the test. Note that the surge values in cities 1 and 2 have been normalized, such that they have 0 mean and standard deviation 1. The true means and std of the surge are of the order of 10cm and 20cm respectively.

The **.npz format**: practically, the input X is encoded in the numpy .npz format and consists of:

- id_sequence: the ids of the sequence
- t_slp: the 40 GMT times at which the sea-level pressure (SLP) fields are given.
- slp: the 40 sea-level pressure (SLP) fields, encoded in images of size (41, 41).
- t_surge1_input: the 10 GMT times at which the surge heights are given in city 1.
- surge1_input: the given surge heights in city 1.
- t_surge2_input: the 10 GMT times at which the surge heights are given in city 2.
- surge2_input: the given surge heights in city 2.
- t_surge1_output: the 10 GMT times at which we must predict surge heights in city 1.
- t_surge2_output: the 10 GMT times at which we must predict surge heights in city 2.

To access for example the training slp, one can use the following:

```python
X_train = np.load('X_train_surge.npz')
slp = X_train['slp']
```

The output Y is encoded in a csv file with the columns:


- id_sequence: the ids of the sequence
- surge1_t{0...9}: the correct surge height in city 1 at time 0 to 9
- surge2_t{0...9}: the correct surge height in city 2 at time 0 to 9

**Submission format** We provided a random submission example. The submission index must be X_test['id_sequence'] and the columns must match those of Y_train_surge.csv: see the notebook in the supplementary files for another submission example. 

## Benchmark description

Analog methods in meteorology are the equivalent of nearest-neighbors predictions in machine learning. It consists in finding a day in the past when the weather scenario looked very similar to the actual weather scenario (an analog scenario). The forecaster would predict that the weather in this forecast will behave the same as it did in the past. Physicists appreciate these methods for their interpretability, as the prediction is explained by a real past situation that they can look at. They can also a posteriori analyze the prediction errors given the difference between the actual scenario and the analog found in the past.

The benchmark we propose here is an analog method. We use the standard L2 metric and look for the closest k=40k=40 scenarios at time t and t - 24h using a K-nearest neighbor search. We then average over these scenarios to get the benchmark. It yields a score of 0.77 on the public test data, meaning that it explained around 23 % of the variance.

We provide in the supplementary files a Jupyter Notebook that implements this benchmark.

## Answer

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [2]:
X_train = np.load(os.path.join('assets', 'X_train_surge_new.npz'))
Y_train = pd.read_csv(os.path.join('assets', 'Y_train_surge.csv'))
X_test = np.load(os.path.join('assets', 'X_test_surge_new.npz'))

In [3]:
s0: tuple[int, ...] = X_train['slp'].shape  # (5599, 40, 41, 41)
s1: tuple[int, int] = Y_train.shape         # (5599, 21)
print(s0, s1)

(5599, 40, 41, 41) (5599, 21)


In [4]:
# create a gif of all 40 images of the first sample
# from https://stackoverflow.com/a/56244531/1164295
def create_gif(images, filename, duration=0.1):
  import imageio
  images = [imageio.imread(image) for image in images]
  imageio.mimsave(filename, images, duration=duration)


# try create assets/images folder
try:
  os.mkdir('assets/images')
except FileExistsError:
  pass

if not os.path.exists(os.path.join('assets', 'images', 'surge_sample.gif')):

  images = []
  for i in range(s0[1]):
    data = X_train['slp'][0, i, :, :]
    plt.imsave(os.path.join('assets', 'images', f'surge_sample_{i}.png'), data, cmap='jet')
    images.append(os.path.join('assets', 'images', f'surge_sample_{i}.png'))

  create_gif(images, os.path.join('assets', 'images', f'surge_sample.gif'), duration=0.1)

In [5]:
# display the gif
from IPython.display import Image
Image(url=os.path.join('assets', 'images', f'surge_sample.gif'), width=500)

In [6]:
# normalize the data
df: np.ndarray = (X_train['slp'] - X_train['slp'].mean()) / X_train['slp'].std()
df = df.reshape((s0[0], s0[1], s0[2] * s0[3]))
del X_train

In [7]:
# create time series model
model = tf.keras.Sequential([
  tf.keras.layers.LSTM(128, input_shape=(s0[1], s0[2] * s0[3])),
  tf.keras.layers.Dense(21)
])

# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [8]:
# train the model
history = model.fit(df, Y_train.values, epochs=10, batch_size=32)


Epoch 1/10
175/175 [==============================] - 5s 11ms/step - loss: 491800.2812
Epoch 2/10
175/175 [==============================] - 2s 11ms/step - loss: 485070.4062
Epoch 3/10
175/175 [==============================] - 2s 10ms/step - loss: 479047.2500
Epoch 4/10
175/175 [==============================] - 2s 10ms/step - loss: 473158.1250
Epoch 5/10
175/175 [==============================] - 2s 10ms/step - loss: 467317.7812
Epoch 6/10
175/175 [==============================] - 2s 10ms/step - loss: 461603.6562
Epoch 7/10
175/175 [==============================] - 2s 11ms/step - loss: 455997.4375
Epoch 8/10
175/175 [==============================] - 2s 10ms/step - loss: 450461.0625
Epoch 9/10
175/175 [==============================] - 2s 10ms/step - loss: 444996.6875
Epoch 10/10
175/175 [==============================] - 2s 10ms/step - loss: 439600.0000
